In [7]:
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import theano.tensor as tt


/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
N = 52
# Number of past years, including the current year for which the antecedent
# conditions are computed
Nlag = 5
Nyrs = 91
Nblocks = 38

# the time block that each month is assigned to such that for 60 different
# months, we are only estimating 38 unique monthly weights
block = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,\
                  18, 19, 20, 21, 22, 23, 24, 25, 25, 26, 26, 27, 27, 28, 28,\
                  29, 29, 30, 30, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34, 34,\
                  34, 35, 35, 35, 36, 36, 36, 37, 37, 37, 38,\
                  38, 38]).reshape(5,12)


df2 = pd.read_csv("data/dataset2.csv", na_values="NA", skiprows=1, sep=" ")
df3 = pd.read_csv("data/dataset3.csv", na_values="NA", skiprows=1, sep=" ")

In [10]:
df2.head()



Year    NPP  YearID  Event1  Event2  Event3  Event4
0  1939   50.0      40     NaN     NaN     NaN     NaN
1  1940    NaN      41    74.0    76.0   106.0    83.0
2  1941   74.0      42     NaN     NaN     NaN     NaN
3  1942  123.0      43     NaN     NaN     NaN     NaN
4  1943  123.0      44    80.0   159.0   137.0   115.0

In [11]:
df3.head()

Year  ppt1  ppt2  ppt3   ppt4  ppt5  ppt6  ppt7  ppt8  ppt9  ppt10  ppt11  \
0  1900  0.25  1.12  1.07  10.56  1.75  0.82  1.14  0.16  1.92   0.15   0.07   
1  1901  0.19  0.38  1.88   3.62  7.47  2.35  0.71  0.72  2.10   0.36   0.02   
2  1902  0.32  0.15  1.50   0.61  2.13  2.43  1.31  0.67  7.12   1.15   0.27   
3  1903  0.16  1.60  1.03   1.50  0.63  2.23  1.06  0.82  0.87   1.70   0.18   
4  1904  0.04  0.34  0.51   0.89  5.37  1.68  1.99  0.71  1.09   0.39   0.00   

   ppt12  
0   0.11  
1   1.37  
2   0.77  
3   0.07  
4   0.12

In [ ]:
with pm.Model() as model:

    # Assign priors to the ANPP regression parameters (covariate effects)
    a = pm.Normal('a', mu=0, sd=1E-07, shape=6)

    # Prior for residual (observation) standard deviation, and compute
    # associated precision
    sig = pm.Uniform('sig', 0, 100)
    tau = tt.pow(sig, -2)

    # Priors for parameters in the Event missing data model:
    mu_ev = pm.Uniform('mu_ev', 0, 500, shape=4)
    sig_ev = pm.Uniform('sig_ev', 0, 500, shape=4)
    tau_ev = tt.pow(sig_ev, -2, shape=4)

    # Some of the precipitation event data are missing, so specify a simple
    # data model for the Event data for the purpose of estimating the
    # missing data:
    Event = pm.Normal('Event', mu=mu_ev, tau=tau_ev, shape=4)

    # Compute sum of deltas (unnormalized weights), to be used to compute
    # the normalized antecedent weights:
    for t in range(Nlag):
        sumD1[t] = sum(delta[:,t])

    sumD = sum(sumD1)

    # Compute the cumulative monthly weights:
    for t in range(Nlag*12):
        cum_weight[t] = sum(weightOrdered[0:t])

    # Compute the month within year weights (alpha’s = wP,m in Box 1 in main
    # text); that is, these weights sum to 1 within each past year
    for m in range(12):
        for t in range(Nlag):
            alpha[m,t] = delta[m,t] / sum(delta[:,t])

    # Compute antecedent precipitation by summing the weighted precipitation
    # variable over months and past years:
    for i in range(Nlag, Nyrs+1):
        for t in range(Nlag):
            ant_sum1[i,t] = sum(antX1[i,:,t])
        antX[i] = sum(ant_sum1[i,:])

    for i in range(N):

        # Define model for latent (mean) NPP; Event[,k] represents the amount
        # of precipitation received in different size classes, where k indexes
        # the even size class (k=1 for < 5 mm; k=2 for 5-15 mm; k=3 for 15-
        # 30 mm; k=4 for >30 mm); convert antecedent precipitation (antX) from
        # inches to mm.
        mu[i] = a[0] + (a[1] * antX[df2.YearID[i]] * 25.4) + \
                (a[2] * df2.Event[i,0]) + (a[3] * df2.Event[i,1]) + \
                (a[4] * df2.Event[i,2]) + (a[5] * df2.Event[i,3])

        # Data model (or likelihood) for the observed NPP data:
        NPP[i] = pm.Normal('NPP', mu=mu[i], tau=tau)

        # Generate “replicated data” to evaluate model fit.
        NPP_rep[i] = pm.Normal('NPP_rep', mu=mu[i], tau=tau)

    # Dirichlet prior for monthly precipitation weights (due to restrictions
    # on when the built-in dirichlet distribution can be used, we are required
    # to use the relationship between the gamma distribution and the dirichlet
    # to assign the dirichlet prior. For each time block into the past, assign
    # the unnormalized weight (deltaX) a gamma(1,1) prior:
    deltaX = pm.Gamma('deltaX', 1, 1, shape=Nblocks)

    for t in range(Nlag):

        # Compute the yearly weights:
        yr_w[t] = sum(weight[:,t])
        alphad[t] = 1

        for m in range(12):

            # Redefine the unnormalized monthly weights to account for
            # post-ANPP # harvest period; i.e., 2nd part involving equals and
            # step functions # sets weight = 0 if in year 1 and in Oct, Nov,
            # or Dec (i.e., post- # ANPP harvest).
            delta[m,t] = (deltaX[block[t,m]]) * \
                            (1 - equals(t,1) * step(m - 9.5))

            # Compute normalized monthly weights, which will be between
            # 0 and 1, and will some to one.
            weight[m,t] = delta[m,t] / sumD

            # Reorder the weights to go from most recent month (Dec of current
            # year) to “oldest” month (Jan at past year = Nlag).
            weightOrdered[t*12 + (12-m+1)] = weight[m,t]

            # For each time into the past, compute the weighted precipitation
            # variable.
            for i in range(Nlag, Nyrs+1):
                antX1[i,m,t] = weight[m,t] * df3.ppt[i-t+1,m]


In [ ]:
pm.traceplot(traces)